# Tensor Flow word2vec skipgram model tutorial

In [1]:
import tensorflow as tf
?tf.placeholder

# The data

In [4]:
import zipfile
import collections
filename = 'text8.zip'

def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words"""
    with zipfile.ZipFile(filename) as f:
        data = f.read(f.namelist()[0]).split()
    return data


words = read_data(filename)

# Build Vocabulary

In [101]:


vocabulary_size = 50000
def build_dataset(words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary


data, count, dictionary, reverse_dictionary = build_dataset(words)
# del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])


Most common words (+UNK) [['UNK', 418391], (b'the', 1061396), (b'of', 593677), (b'and', 416629), (b'one', 411764)]
Sample data [5235, 3081, 12, 6, 195, 2, 3136, 46, 59, 156]


In [31]:
counts = 0
for key,val in reverse_dictionary.items():
    counts+=1
    print(key,val)
    if counts == 20:break

0 UNK
1 b'the'
2 b'of'
3 b'and'
4 b'one'
5 b'in'
6 b'a'
7 b'to'
8 b'zero'
9 b'nine'
10 b'two'
11 b'is'
12 b'as'
13 b'eight'
14 b'for'
15 b's'
16 b'five'
17 b'three'
18 b'was'
19 b'by'


In [32]:
counts = 0
for key,val in dictionary.items():
    counts+=1
    print(key,val)
    if counts == 20:break

b'recitation' 24400
b'hungarians' 16151
b'timpani' 47037
b'matrimonial' 47882
b'madeleine' 22154
b'mobilizing' 47136
b'plundered' 24742
b'emancipated' 36767
b'vapor' 10572
b'lignite' 30527
b'slated' 17679
b'discogs' 43957
b'grenades' 21248
b'distilled' 12118
b'this' 33
b'trivium' 33558
b'homered' 41479
b'swam' 36768
b'swim' 12801
b'detectives' 21083


# Create the training batches

In [102]:
import random
data_index = 0
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels

In [103]:
import numpy as np

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
    print(batch[i], '->', labels[i, 0])
    print(reverse_dictionary[batch[i]], '->', reverse_dictionary[labels[i, 0]])

3081 -> 12
b'originated' -> b'as'
3081 -> 5235
b'originated' -> b'anarchism'
12 -> 3081
b'as' -> b'originated'
12 -> 6
b'as' -> b'a'
6 -> 12
b'a' -> b'as'
6 -> 195
b'a' -> b'term'
195 -> 2
b'term' -> b'of'
195 -> 6
b'term' -> b'a'


In [110]:
np.unique(labels)

array([   2,    6,   12,  195, 3081, 5235], dtype=int32)

# Parameters

In [104]:
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1  # How many words to consider left and right.
num_skips = 2  # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.

valid_size = 16  # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64  # Number of negative examples to sample.

# The neural netwrok

In [105]:
import math
graph = tf.Graph()

with graph.as_default():
    # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    with tf.device('/cpu:0'):
        # Look up embeddings for inputs.
        embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

        # Construct the variables for the NCE loss
        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size],
                                stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    loss = tf.reduce_mean(
        tf.nn.nce_loss(nce_weights, nce_biases, embed, train_labels,
                       num_sampled, vocabulary_size))

    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
        normalized_embeddings, valid_dataset)
    similarity = tf.matmul(
        valid_embeddings, normalized_embeddings, transpose_b=True)


# Start training

In [106]:
from six.moves import xrange
num_steps = 10001

with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    tf.initialize_all_variables().run()
    print("Initialized")

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(
            batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print("Average loss at step ", step, ": ", average_loss)
            average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = "Nearest to %s:" % valid_word
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = "%s %s," % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()



Initialized
Average loss at step  0 :  279.873657227
Nearest to b'would': b'goya', b'arrest', b'unvoiced', b'beget', b'canaries', b'communicated', b'fgth', b'soares',
Nearest to b'united': b'basics', b'plebeian', b'keane', b'sunny', b'autocad', b'repel', b'penetrative', b'proclaims',
Nearest to b'world': b'caso', b'descended', b'ostracism', b'aoraki', b'trianon', b'storey', b'edifice', b'focusing',
Nearest to b'state': b'unclean', b'grievances', b'entertainer', b'calamity', b'determinative', b'footprints', b'pairwise', b'sucre',
Nearest to UNK: b'diem', b'atop', b'steiner', b'synaptic', b'pyroxene', b'sit', b'horner', b'accommodates',
Nearest to b'been': b'exactly', b'almonds', b'chicxulub', b'transnational', b'sent', b'cheng', b'martial', b'madness',
Nearest to b'such': b'tritone', b'work', b'bucks', b'granger', b'collectors', b'belizean', b'pakistan', b'cerebellum',
Nearest to b'used': b'knowing', b'wishes', b'refrains', b'schwann', b'overlooks', b'boulders', b'destination', b'malate

In [112]:
final_embeddings[0][:]

array([ 0.10907257,  0.03734688,  0.00060537, -0.02426756,  0.06078653,
       -0.02148508,  0.02069261,  0.01886133, -0.25833702, -0.01175279,
        0.00349099, -0.00745373,  0.09060957, -0.10150829,  0.02241914,
        0.02863487,  0.04062018,  0.04342534, -0.01792149,  0.08217603,
        0.05108473,  0.12532884,  0.065133  , -0.07115991, -0.03451871,
        0.07948707,  0.06223791,  0.22644682, -0.12640584,  0.04837745,
       -0.04203285, -0.18494421,  0.10877381,  0.14104903,  0.05339127,
        0.01725111,  0.03982851, -0.07162225,  0.06400462, -0.0505368 ,
        0.03836646, -0.12199939, -0.11222667, -0.11943715,  0.01042285,
       -0.06808503,  0.12355681, -0.0086215 ,  0.05091517, -0.11755998,
       -0.13493635, -0.07338495, -0.08532739,  0.03783849, -0.04994537,
        0.01420888,  0.19505773,  0.25991973,  0.13823561,  0.09248477,
       -0.16074032,  0.10206009, -0.12440103,  0.03316716, -0.06354502,
        0.02777245, -0.07848439, -0.03084324,  0.06095168, -0.09

In [120]:
(-sim[0, :])

array([-0.19808948, -0.30545813, -0.37229383, ...,  0.00951003,
        0.05122453, -0.23305655], dtype=float32)

In [125]:
(-sim[0, :]).argsort()

array([   74, 49145,  4753, ..., 49803, 32074, 27924])

In [95]:
reverse_dictionary[nearest[0]]

b'medians'

In [123]:
valid_examples

array([74, 86, 71, 94,  0, 54, 55, 59, 69, 78, 12, 76,  9, 63, 89, 52])

In [117]:
[reverse_dictionary[i] for i in valid_examples]

[b'would',
 b'united',
 b'world',
 b'state',
 'UNK',
 b'been',
 b'such',
 b'used',
 b'may',
 b'between',
 b'as',
 b'no',
 b'nine',
 b'into',
 b'called',
 b'most']

In [133]:
sim2 = final_embeddings.dot(final_embeddings.T)

In [135]:
i = 74
valid_word2 = reverse_dictionary[i]
valid_word2

b'would'

In [136]:
nearest = (-sim2[i, :]).argsort()[1:top_k + 1]

In [141]:
log_str = "Nearest to %s:" % valid_word2
for k in xrange(top_k):
    close_word = reverse_dictionary[nearest[k]]
    log_str = "%s %s," % (log_str, close_word)
print(log_str)

Nearest to b'would': b'victoriae', b'arrest', b'gland', b'seek', b'kennedy', b'penguins', b'council', b'reginae',
